In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

consistent_loinc = {
    "C_reactive_protein_CRP_Normal_Sensitivity": "C-reactive protein (Normal Sensitivity) (mg/dL)",
    "creatinine": "Creatinine (mg/dL)",
    "Ferritin": "Ferritin (ng/mL)",
    "D_dimer": "D-dimer (ng/mL)",
    "albumin": "Albumin (g/dL)",        

    "Fibrinogen": "Fibrinogen (mg/dL)",
    "alanine_aminotransferase_ALT": "Alanine aminotransferase (U/L)",
    "aspartate_aminotransferase_AST": "Aspartate aminotransferase (U/L)",
    "total_bilirubin": "Total bilirubin (mg/dL)",
    "lactate_dehydrogenase_LDH": "Lactate dehydrogenase (U/L)",
    "cardiac_troponin_High_Sensitivity": "Cardiac troponin High Sensitivity (ng/mL)",
    "cardiac_troponin_Normal_Sensitivity": "Cardiac troponin Normal Sensitivity (ng/mL)",
    "prothrombin_time_PT": "Prothrombin time (s)",
    "white_blood_cell_count_Leukocytes": "White blood cell count (10*3/uL)",
    "lymphocyte_count": "Lymphocyte count (10*3/uL)",
    "neutrophil_count": "Neutrophil count (10*3/uL)",
    "procalcitonin": "Procalcitonin (ng/mL)",
}

# BETA HEATMAP

In [ ]:
df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.beta.R1.toShare.csv"))

#df = df.drop(columns=['Unnamed: 0'])

df.siteid = df.siteid.apply(lambda x: x.replace('meta-', '').replace('meta', 'All'))

consistent_c = {
    'age00to25': '0-25',
    'age26to49': '26-49',
    'age70to79': '70-79',
    'age80plus': '80+',
    'sexfemale': 'Female',
    'raceBlack': 'Black',
    'raceAsian': 'Asian',
    'raceHispanic.and.Other': 'Hispanic and Other',
    'albumin': 'Albumin',
    "AST": 'AST',
    "ALT": 'ALT',
    "creatinine": 'Creatinine',
    'CRP': 'CRP',
    'DD': 'D-dimer',
    'TB': 'Total bilirubin',
    "LYM": 'Lymphocyte Count',
    "neutrophil_count" : 'Neutrophil Count',
    "WBC" : 'White Blood Cell',
    'charlson_score': 'Charlson Score'
}

idf = pd.read_csv(join('..', 'data', "SiteID_MAP_Phase1.1.csv"), sep=",", header=0)
mapping = dict(zip(idf.siteid, idf.fakeid))

# Missing key-value pairs
mapping['All'] = 'All'
mapping['USA'] = 'USA'
mapping['France'] = 'France'
mapping['Germany'] = 'Germany'

# df = df[~df.variable.str.contains('mis_')]

# print(df.variable.unique().tolist())
df.variable = df.variable.apply(lambda x: consistent_c[x])
df.siteid = df.siteid.apply(lambda x: mapping[x])

print(df.variable)
#print(df.siteid)
# df = df.rename(columns={"site.from": "from", 'site.to': 'to', 'N.from': 'from_size'})

# site_to_contry = {
#     'APHP': 'France',
#     'FRBDX': 'France',
#     'UKFR': 'France',
#     'BIDMC': 'Germany',
#     'MGB': 'USA',
#     'NWU': 'USA', 
#     'UCLA': 'USA', 
#     'UMICH': 'USA', 
#     'UPENN': 'USA', 
#     'UPITT': 'USA', 
#     'VA1': 'USA', 
#     'VA2': 'USA', 
#     'VA3': 'USA', 
#     'VA4': 'USA', 
#     'VA5': 'USA',
#     'all': 'Meta',
#     'US': 'Meta',
#     'France': 'Meta',
#     'Germany': 'Meta',
# }

# df['country'] = df['from'].apply(lambda x: site_to_contry[x])
# df['to-country'] = df['to'].apply(lambda x: site_to_contry[x])

# df['from'] = df['from'].apply(lambda x: f"{x} →")
# df['to'] = df['to'].apply(lambda x: f"→ {x.replace('all', 'All').replace('US', 'USA')}") # ←


# countries = ['USA', 'France', 'Germany'] # , 'ITALY']
# country_colors = ['#D45E00', '#0072B2', '#029F73'] # , '#B2AA2F']

# # df = pd.melt(df, id_vars=['siteid', 'to'], var_name='day', value_name='value')

unique_sites = df.siteid.unique().tolist()
print(unique_sites)


unique_vars = df.variable.unique().tolist()
print(unique_vars)

df['meta'] = df.siteid.apply(lambda x: True if (x == 'All') or (x == 'USA') or (x == 'France') or (x == 'Germany') else False)

# df.beta = df.beta.apply(lambda x: -3 if x <= -3 else 3 if x >= 3 else x)

# df[df.variable == 'mis_albumin']
df

In [ ]:
site_sort = ['SITE1', 'SITE2', 'SITE3', 'SITE4', 'SITE5', 'SITE6', 'SITE7', 'SITE8',  'SITE9',    'SITE10', 'SITE11', 'SITE12', 'SITE13','SITE14','SITE15', 'All', 'USA', 'France', 'Germany']
var_sort = [
    '0-25', '26-49', '70-79', '80+', 'Female', 'Black', 'Asian', 'Hispanic and Other', 'Albumin', 'ALT', 'AST', 'Creatinine', 'CRP', 'D-dimer', 'Lymphocyte Count', 'Neutrophil Count', 'Total bilirubin', 'White Blood Cell', 'Charlson Score'
]

def plot(df=None):
    d = df.copy()
    
    # Make sure not to draw black colors.
    d.beta = d.beta.apply(lambda x: 3 if x > 3 else -3 if x < -3 else x)
    
    plot = alt.Chart(
        d[d['meta'] == False]
    ).mark_square(
        opacity=1
    ).encode(
        y=alt.Y("variable:N", title='Variable', axis=alt.Axis(labelAngle=0, domain=True, orient='left'), scale=alt.Scale(), sort=unique_vars),
        x=alt.X("siteid:N", title='Site', axis=alt.Axis(labelAngle=-55, domain=True, orient='top', titleY=-60), scale=alt.Scale(), sort=site_sort),
        size=alt.value(1100),
        color=alt.Color("beta:Q", title='Coefficient', scale=alt.Scale(scheme='pinkyellowgreen', domain=[-3, 3]), legend=alt.Legend(direction='horizontal', gradientLength=440, gradientThickness=30)),
    ).properties(
        width=510,
        height=610
    )
    
    plot2 = alt.Chart(
        d[d['meta'] == True]
    ).mark_square(
        opacity=1
    ).encode(
        y=alt.Y("variable:N", title=None, axis=alt.Axis(labelAngle=0, domain=False, orient='left', labels=False), scale=alt.Scale(), sort=unique_vars),
        x=alt.X("siteid:N", title='Meta Analysis', axis=alt.Axis(labelAngle=-55, domain=True, orient='top', titleY=-60), scale=alt.Scale(), sort=site_sort),
        size=alt.value(1100),
        color=alt.Color("beta:Q", title='Coefficient', scale=alt.Scale(scheme='pinkyellowgreen', domain=[-3, 3]), legend=alt.Legend(direction='horizontal', gradientLength=440, gradientThickness=30)),
    ).properties(
        width=135,
        height=610
    )
    
    return (plot | plot2)

res = plot(df)

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    subtitle_font_size=18,
#     legend_stroke_color='white'
)

res
save(res,join("..", "result", "R1-beta-matrix.png"), scalefactor=8.0)

